In [3]:
import os
from pathlib import Path

appdata = Path(os.getenv("APPDATA")) / "Pandora" / "Pipesim Pilot"
appdata.mkdir(parents=True, exist_ok=True)
appdata

WindowsPath('C:/Users/IDM252577/AppData/Roaming/Pandora/Pipesim Pilot')